In [1]:
import pandas as pd

In [2]:
import os
os.listdir('data')

['202005_petroleo_cnmc.xlsx',
 'cnmc_goa_gna.xlsx',
 'Estadisticas_petroleo_mayo_2020.xlsx',
 '~$Estadisticas_petroleo_mayo_2020.xlsx']

In [3]:
INPATH = "data/"
INFILE1 = "202005_petroleo_cnmc.xlsx"

In [4]:
OUTPATH = 'data/'
OUTFILE1 = 'cnmc_goa_gna.xlsx'
OUTFILE2 = 'cnmc_prov_month_prices.xlsx'

In [5]:
START_DT = '2003-01-01'
END_DT = '2020-03-01'

In [6]:
month_parser = ["enero",
"febrero",
"marzo",
"abril",
"mayo",
"junio",
"julio",
"agosto",
"septiembre",
"octubre",
"noviembre",
"diciembre",
]

In [7]:
def create_date(yr, mon, day):
    return pd.to_datetime(10000*yr.astype(float)+100*mon.astype(float)+1, format='%Y%m%d')

# Load Data

In [8]:
cons_renaming = {'GASÓLEO A': 'cons_GOA',
           'GASOLINA  AUTO. S/PB 95 I.O.': 'cons_GNA95',
           'GASOLINA  AUTO. S/PB 98 I.O.': 'cons_GNA98'}

cons_select_cols = ['cons_GOA', 'cons_GNA95', 'cons_GNA98']

In [9]:
cons = pd.read_excel(INPATH + INFILE1, sheet_name ='Con')
cons = cons[cons['MES']!='ANUAL']
cons['MES'] = cons['MES'].replace(month_parser, range(1,len(month_parser)+1)).astype(int)
cons['Date'] = cons[['AÑO', 'MES']].apply(lambda x: create_date(yr=x[0], mon=x[1], day=1), axis=1)

cons.rename(inplace=True, columns = cons_renaming)
cons.set_index('Date', inplace=True)
cons = cons.loc[START_DT:END_DT, cons_select_cols].copy()  #kts
pd.concat([cons.head(3), cons.tail(3)], axis=0)

,cons_GOA,cons_GNA95,cons_GNA98
Date,,,
2003-01-01,1.608696e+06,439963.562137,60640.977069
2003-02-01,1.599444e+06,412083.235859,57829.011088
2003-03-01,1.715755e+06,467030.776111,65063.645973
2019-12-01,1.840063e+06,410458.860000,34767.810000
2020-01-01,1.789099e+06,380321.020000,29967.040000
2020-02-01,1.804107e+06,380494.340000,29840.130000


In [10]:
cot_renaming = {'GASÓLEO A': 'spot_GOA',
            'GASOLINA': 'spot_GNA',
            'BRENT': 'spot_Brent'}

cot_select_cols = ['spot_GOA', 'spot_GNA', 'spot_Brent']

In [11]:
cot = pd.read_excel(INPATH + INFILE1, sheet_name ='Cot')
cot = cot.rename( columns = cot_renaming).rename( columns = {'MES': 'Date'})
cot.set_index('Date', inplace=True)

cot = cot.loc[START_DT:END_DT, cot_select_cols].copy()
cot.head()

,spot_GOA,spot_GNA,spot_Brent
Date,,,
2007-01-01,395.853111,383.742561,42.207328
2007-02-01,416.684195,422.002876,44.977131
2007-03-01,434.833042,476.939514,47.164810
2007-04-01,460.730439,530.646820,50.086227
2007-05-01,468.679841,570.140658,50.209746


In [12]:
prices_renaming = {'GASÓLEO A': 'price_GOA',
           'GASOLINA  AUTO. S/PB 95 I.O.': 'price_GNA95',
           'GASOLINA  AUTO. S/PB 98 I.O.': 'price_GNA98'}

prices_select_cols = ['price_GOA', 'price_GNA95', 'price_GNA98']

In [13]:
prices = pd.read_excel(INPATH + INFILE1, sheet_name ='PVP')

prices['MES'] = prices['MES'].replace(month_parser, range(1,len(month_parser)+1)).astype(int)
prices['Date'] = prices[['AÑO', 'MES']].apply(lambda x: create_date(yr=x[0], mon=x[1], day=1), axis=1)
prices.rename(inplace=True, columns = prices_renaming)

prices.set_index('Date', inplace=True)
prices = prices.loc[START_DT:END_DT, prices_select_cols].copy()
pd.concat([prices.head(3), prices.tail(3)], axis=0)

,price_GOA,price_GNA95,price_GNA98
Date,,,
2007-01-01,0.883903,0.940323,1.052226
2007-02-01,0.888000,0.946250,1.057107
2007-03-01,0.908194,0.991258,1.100032
2020-01-01,1.241000,1.316000,1.449000
2020-02-01,1.201000,1.295000,1.432000
2020-03-01,1.114000,1.209000,1.349000


In [14]:
comb = pd.concat([cons, cot, prices],axis=1).dropna()
pd.concat([comb.head(3), comb.tail(3)], axis=0)

,cons_GOA,cons_GNA95,cons_GNA98,spot_GOA,spot_GNA,spot_Brent,price_GOA,price_GNA95,price_GNA98
Date,,,,,,,,,
2007-01-01,2.028811e+06,459030.653532,63019.746212,395.853111,383.742561,42.207328,0.883903,0.940323,1.052226
2007-02-01,1.979790e+06,425537.713411,60966.501392,416.684195,422.002876,44.977131,0.888000,0.946250,1.057107
2007-03-01,2.314793e+06,503274.794375,74170.996622,434.833042,476.939514,47.164810,0.908194,0.991258,1.100032
2019-12-01,1.840063e+06,410458.860000,34767.810000,546.510439,542.907143,58.743030,1.224000,1.303000,1.436000
2020-01-01,1.789099e+06,380321.020000,29967.040000,517.998215,534.999139,57.422037,1.241000,1.316000,1.449000
2020-02-01,1.804107e+06,380494.340000,29840.130000,460.429849,496.110729,50.873453,1.201000,1.295000,1.432000


In [15]:
cons_renaming

{'GASÓLEO A': 'cons_GOA',
 'GASOLINA  AUTO. S/PB 95 I.O.': 'cons_GNA95',
 'GASOLINA  AUTO. S/PB 98 I.O.': 'cons_GNA98'}

In [16]:
prices_prov = pd.read_excel(INPATH + INFILE1, sheet_name ='PVP_Prov')
prices_prov.rename(inplace=True, columns=prices_renaming)
prices_prov = prices_prov[prices_prov['PROVINCIA']!='TOTAL'].copy()
prices_prov['MES'] = prices_prov['MES'].replace(month_parser, range(1,len(month_parser)+1)).astype(int)
prices_prov['Date'] = prices_prov[['AÑO', 'MES']].apply(lambda x: create_date(yr=x[0], mon=x[1], day=1), axis=1)

# Train/Test split

In [17]:
split_col = 'split'
test_size = 12

In [18]:
idx_time = comb.index
idx_train, idx_test = idx_time[:-test_size], idx_time[-test_size:]

In [19]:
comb[split_col] = 'train'
comb.loc[idx_test, split_col] = 'test'
comb[split_col].value_counts()

train    146
test      12
Name: split, dtype: int64

In [20]:
last_date = prices_prov['Date'].max()
TRAIN_END_DT = last_date - pd.DateOffset(months=test_size)
TRAIN_END_DT

Timestamp('2019-04-01 00:00:00')

In [21]:
prices_prov[split_col] = 'train'
prices_prov.loc[prices_prov['Date'] >TRAIN_END_DT, split_col] = 'test'
prices_prov[split_col].value_counts()

train    7696
test      624
Name: split, dtype: int64

In [22]:
comb.to_excel(OUTPATH + OUTFILE1)

In [23]:
prices_prov.to_excel(OUTPATH + OUTFILE2)